# Introduction


**What?** Creating a TorchScript



# Import modules

In [1]:
import torch.nn as nn
import torch

# What is a Torch script?


- TorchScript creates serializable and optimizable versions of models from PyTorch code. 
- It is possible to create a TorchScript in two ways: 
    - tracing, or 
    - using a script compiler. 



# First method to create a serialised version of the code

In [ ]:
"""
We used the tracing method to create a TorchScript. We defined a simple module called MyCell to be converted
into Torchscript and created two sample tensors called x and h to be passed into the forward method of the 
network module. Then, we used jit.trace to trace the Python code and create the TorchScript.
"""

In [ ]:
"""
We used the tracing method to create a TorchScript. We defined a simple module called MyCell to be converted
into Torchscript and created two sample tensors called x and h to be passed into the forward method of the 
network module. Then, we used jit.trace to trace the Python code and create the TorchScript.
"""

In [ ]:
"""
TorchScript records its definitions in an intermediate representation (referred to as a graph in deep learning).
Then, we examined the graph with the .graph property and generated a more readable version using .code, which is
the Python syntax interpretation of the code.
"""

In [2]:
class MyCell(torch.nn.Module):
    def __init__(self):
        super(MyCell, self).__init__()
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.linear(x) + h)
        return new_h

In [3]:
my_cell = MyCell()

In [4]:
x, h = torch.rand(4, 4), torch.rand(4, 4)

In [5]:
traced_cell = torch.jit.trace(my_cell, (x, h))

In [6]:
traced_cell

MyCell(
  original_name=MyCell
  (linear): Linear(original_name=Linear)
)

In [7]:
traced_cell(x, h)

tensor([[-0.0338,  0.2625,  0.7189,  0.2757],
        [ 0.2686, -0.2427,  0.6835, -0.2930],
        [ 0.0460,  0.6556,  0.4867, -0.2667],
        [ 0.2213, -0.2658,  0.6971, -0.3194]], grad_fn=<TanhBackward>)

In [8]:
traced_cell.graph

graph(%self.1 : __torch__.MyCell,
      %input : Float(4:4, 4:1, requires_grad=0, device=cpu),
      %h : Float(4:4, 4:1, requires_grad=0, device=cpu)):
  %18 : __torch__.torch.nn.modules.linear.Linear = prim::GetAttr[name="linear"](%self.1)
  %20 : Tensor = prim::CallMethod[name="forward"](%18, %input)
  %12 : int = prim::Constant[value=1]() # <ipython-input-2-c6e2cd8665ee>:7:0
  %13 : Float(4:4, 4:1, requires_grad=1, device=cpu) = aten::add(%20, %h, %12) # <ipython-input-2-c6e2cd8665ee>:7:0
  %14 : Float(4:4, 4:1, requires_grad=1, device=cpu) = aten::tanh(%13) # <ipython-input-2-c6e2cd8665ee>:7:0
  return (%14)

In [9]:
# For a readable version of the above cell, we can use the following command
traced_cell.code

'def forward(self,\n    input: Tensor,\n    h: Tensor) -> Tensor:\n  _0 = torch.add((self.linear).forward(input, ), h, alpha=1)\n  return torch.tanh(_0)\n'

# Second method to create a serialised version of the code

In [ ]:
"""
We explored the next method of creating a TorchScript, which was by using a script compiler
"""

In [10]:
class MyDecisionGate(torch.nn.Module):
    def forward(self, x):
        if x.sum() > 0:
            return x
        else:
            return -x

In [11]:
class MyCell(torch.nn.Module):
    def __init__(self, dg):
        super(MyCell, self).__init__()
        self.dg = dg
        self.linear = torch.nn.Linear(4, 4)

    def forward(self, x, h):
        new_h = torch.tanh(self.dg(self.linear(x)) + h)
        return new_h

In [ ]:
"""
With the tracing method, we lose the flow of control since, with tracing, we ran the code, recorded the 
operations, and constructed a ScriptModule object that erases things such as control flow.
"""

In [12]:
my_cell = MyCell(MyDecisionGate())
traced_cell = torch.jit.trace(my_cell, (x, h))

<ipython-input-10-830ab158f335>:3: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.sum() > 0:


In [17]:
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tensor:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  return torch.tanh(torch.add(_0, h, alpha=1))



In [ ]:
"""
Due to this, we use jit.script, which preserves the control flow.
"""

In [18]:
scripted_gate = torch.jit.script(MyDecisionGate())

In [19]:
my_cell = MyCell(scripted_gate)

In [20]:
traced_cell = torch.jit.script(my_cell)
print(traced_cell.code)

def forward(self,
    x: Tensor,
    h: Tensor) -> Tensor:
  _0 = (self.dg).forward((self.linear).forward(x, ), )
  return torch.tanh(torch.add(_0, h, alpha=1))



In [ ]:
"""
When we printed the TorchScript code using print(traced_cell.code), we saw that the flow of control was still 
preserved.
"""

# References


- Jibin Mathew, PyTorch Artificial Intelligence Fundamentals
- https://github.com/jibinmathew69

